# Average speeds on segments as time-series

## Exploratory: aggregate to route-direction-stop columns 

**Metrics**
* count number of speed values by stop (check for equality against number of service dates) -- how many do we get right vs have dupes
   * if `n_speed_values = n_dates`, we are tracking the same segment over time
   * if `n_speed_values > n_dates`, there are duplicate segments that are getting counted for each date
* how many geometry values are present for a segment?
   * anything > 1 means there's a ~dupe segment, or a segment cut slightly differently -- why does this happen, do the segments differ significantly or not really?

With time-series, we can now track how well `route-dir-stop_pair` is at capturing the same geographic entity over time. Aggregations are done for each day, and we don't really know how well they are performing across time. 

Goal is to say: these are the average speeds on this segment over this period of time.

Route-direction is more stable than shape (definitely trip) grain. With GTFS Digest, we saw that around the 6 month timeframe, even `route_id` starts changing. We do harmonize across time with route names, so let's see how well we're doing now at tracking the same segment over time.

In [1]:
import branca
import folium
import geopandas as gpd
import pandas as pd

import segment_time_series_utils as ts_utils

In [2]:
df = ts_utils.route_segment_speeds_ts(
    filters = [[("time_period", "==", "all_day")]],
    columns = ts_utils.route_dir_stop_cols + [
        "service_date", "p50_mph", "geometry"]
)

df2 = ts_utils.count_time_series_values_by_route_direction_stop(
    df, ts_utils.route_dir_stop_cols)

In [3]:
df.service_date.nunique()

18

In [4]:
df2[df2.n_speed_values == df2.n_dates].shape[0] / df2.shape[0]

0.9539929595635667

In [5]:
df2[df2.n_speed_values > df2.n_dates].shape[0] / df2.shape[0]

0.04600704043643334

* 95% rows are getting it right, 5% has dupes (speed counts are more than the dates, which means there are some duplicated counts)
* about 55% of segment geometries stay stable across 17 months...85% have 1 or 2 geometries over this period.
* geometries may need to get harmonized or not...if they are slightly differing, we may not care
* the different segments getting cut would reflect the fact that shape changes. `stop_segments` already uses 1 shape for each route-direction, so intra-date-variation is already at a minimum....yet over 17 months, we are seeing this aggregation need further reducing of variation.

In [6]:
df2.n_geometry.value_counts(normalize=True)

1     0.539810
2     0.303799
3     0.137809
4     0.013666
6     0.002507
5     0.001894
7     0.000207
8     0.000170
9     0.000088
10    0.000041
13    0.000005
12    0.000005
Name: n_geometry, dtype: float64

In [7]:
dupes = df2[df2.n_geometry>=3].reset_index(drop=True)
dupes.shape

(33942, 10)

In [8]:
def plot_segment_variations(
    df: gpd.GeoDataFrame, 
    dupes_df: pd.DataFrame,
    row_value: int
):
    """
    Make a map of the variations of segments so we can see 
    just how similar or different they look.
    """
    subset_gdf = pd.merge(
        df,
        dupes_df.iloc[row_value : row_value + 1],
        on = ts_utils.route_dir_stop_cols,
        how = "inner"
    )[ts_utils.route_dir_stop_cols + ["geometry"]
    ].drop_duplicates().reset_index(drop=True)
    
    display(subset_gdf)
    
    subset_gdf = subset_gdf.assign(
        obs = subset_gdf.index
    )
    
    m = subset_gdf[subset_gdf.obs==0].explore(
        "obs",
        tiles = "CartoDB Positron",
        cmap = branca.colormap.step.Set1_03, 
        name = "obs 0",
        width = "75%", height = "75%"
    )
    
    for idx, value in enumerate(subset_gdf[subset_gdf.obs > 0].obs.unique()):
        m = subset_gdf[subset_gdf.obs == value].explore(
            m=m, cmap=branca.colormap.step.Set1_03, name = f"obs {value}"
        )

    folium.LayerControl().add_to(m)
    
    display(m)

### 3 variations

Pick about 4 examples randomly

In [9]:
subset_df = dupes[dupes.n_geometry == 3].reset_index(drop=True)

for v in range(
    0, len(subset_df), int(len(subset_df)//4)
):
    plot_segment_variations(df, subset_df, v)

,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40092__41042,Bridgeway & Nevada St__Bridgeway & Coloma St,"LINESTRING (-122.49913 37.86532, -122.49925 37..."
1,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40092__41042,Bridgeway & Nevada St__Bridgeway & Coloma St,"LINESTRING (-122.49913 37.86532, -122.49925 37..."
2,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40092__41042,Bridgeway & Nevada St__Bridgeway & Coloma St,"LINESTRING (-122.49886 37.86505, -122.49913 37..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,207,1,14008__13994,Van Ness / Imperial__Van Ness / 120th,"LINESTRING (-118.31777 33.93070, -118.31777 33..."
1,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,207,1,14008__13994,Van Ness / Imperial__Van Ness / 120th,"LINESTRING (-118.31777 33.93070, -118.31777 33..."
2,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,207,1,14008__13994,Van Ness / Imperial__Van Ness / 120th,"LINESTRING (-118.31777 33.93070, -118.31777 33..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,94,0,15864__18018,Avenue 26 / Idell__San Fernando / Pepper,"LINESTRING (-118.22505 34.08621, -118.22505 34..."
1,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,94,0,15864__18018,Avenue 26 / Idell__San Fernando / Pepper,"LINESTRING (-118.22505 34.08621, -118.22505 34..."
2,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,94,0,15864__18018,Avenue 26 / Idell__San Fernando / Pepper,"LINESTRING (-118.22505 34.08621, -118.22505 34..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,dcb6fb26f2d08393f9b0b1c14dc8775c,Bay Area 511 Tri-Valley Wheels Schedule,all_day,607,0,882382__882343,Regency Dr & Belvedere Ct__Jorgensen Ln & Lyle...,"LINESTRING (-121.91156 37.66124, -121.91045 37..."
1,dcb6fb26f2d08393f9b0b1c14dc8775c,Bay Area 511 Tri-Valley Wheels Schedule,all_day,607,0,882382__882343,Regency Dr & Belvedere Ct__Jorgensen Ln & Lyle...,"LINESTRING (-121.91156 37.66124, -121.91045 37..."
2,dcb6fb26f2d08393f9b0b1c14dc8775c,Bay Area 511 Tri-Valley Wheels Schedule,all_day,607,0,882382__882343,Regency Dr & Belvedere Ct__Jorgensen Ln & Lyle...,"LINESTRING (-121.91151 37.66114, -121.91091 37..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,School 256,0,60750__64128,Tamien Station__Alma & Lick,"LINESTRING (-121.88475 37.31102, -121.88483 37..."
1,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,School 256,0,60750__64128,Tamien Station__Alma & Lick,"LINESTRING (-121.88475 37.31102, -121.88483 37..."
2,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,School 256,0,60750__64128,Tamien Station__Alma & Lick,"LINESTRING (-121.88475 37.31102, -121.88483 37..."


### 4 variations

In [10]:
subset_df = dupes[dupes.n_geometry == 4].reset_index(drop=True)

for v in range(
    0, subset_df.shape[0], int(subset_df.shape[0]//4)
):
    plot_segment_variations(df, subset_df, v)

,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40130__40182,E Blithedale Ave & Kipling Dr__Reed Blvd & Red...,"LINESTRING (-122.51905 37.90349, -122.51871 37..."
1,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40130__40182,E Blithedale Ave & Kipling Dr__Reed Blvd & Red...,"LINESTRING (-122.51932 37.90357, -122.51932 37..."
2,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40130__40182,E Blithedale Ave & Kipling Dr__Reed Blvd & Red...,"LINESTRING (-122.51905 37.90349, -122.51871 37..."
3,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40130__40182,E Blithedale Ave & Kipling Dr__Reed Blvd & Red...,"LINESTRING (-122.51932 37.90357, -122.51905 37..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,910,0,3153__3124,Beacon / 1st__Harbor Beacon Park and Ride,"LINESTRING (-118.28042 33.74364, -118.28042 33..."
1,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,910,0,3153__3124,Beacon / 1st__Harbor Beacon Park and Ride,"LINESTRING (-118.28042 33.74329, -118.28042 33..."
2,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,910,0,3153__3124,Beacon / 1st__Harbor Beacon Park and Ride,"LINESTRING (-118.28042 33.74364, -118.28042 33..."
3,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,910,0,3153__3124,Beacon / 1st__Harbor Beacon Park and Ride,"LINESTRING (-118.28042 33.74364, -118.28042 33..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,8eecb796518dafd3c1b971a99f8b8252,Victor Valley GMV Schedule,all_day,3206,1,3122146__3117300,Hook Blvd EB & Rosemary Dr__Hook Blvd EB & Arl...,"LINESTRING (-117.34671 34.52124, -117.34085 34..."
1,8eecb796518dafd3c1b971a99f8b8252,Victor Valley GMV Schedule,all_day,3206,1,3122146__3117300,Hook Blvd EB & Rosemary Dr__Hook Blvd EB & Arl...,"LINESTRING (-117.34671 34.52124, -117.34085 34..."
2,8eecb796518dafd3c1b971a99f8b8252,Victor Valley GMV Schedule,all_day,3206,1,3122146__3117300,Hook Blvd EB & Rosemary Dr__Hook Blvd EB & Arl...,"LINESTRING (-117.34671 34.52124, -117.34085 34..."
3,8eecb796518dafd3c1b971a99f8b8252,Victor Valley GMV Schedule,all_day,3206,1,3122146__3117300,Hook Blvd EB & Rosemary Dr__Hook Blvd EB & Arl...,"LINESTRING (-117.34671 34.52124, -117.34085 34..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,22,1,1027__1028,Alessandro FS Cannon__Alessandro FS Canyon Crest,"LINESTRING (-117.33638 33.92354, -117.33643 33..."
1,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,22,1,1027__1028,Alessandro FS Cannon__Alessandro FS Canyon Crest,"LINESTRING (-117.33638 33.92354, -117.33643 33..."
2,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,22,1,1027__1028,Alessandro FS Cannon__Alessandro FS Canyon Crest,"LINESTRING (-117.33638 33.92354, -117.33657 33..."
3,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,22,1,1027__1028,Alessandro FS Cannon__Alessandro FS Canyon Crest,"LINESTRING (-117.33626 33.92342, -117.33638 33..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,Rapid 500,0,65850__60413,Santa Clara & 2nd__Santa Clara & 6th,"LINESTRING (-121.88947 37.33669, -121.88844 37..."
1,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,Rapid 500,0,65850__60413,Santa Clara & 2nd__Santa Clara & 6th,"LINESTRING (-121.89017 37.33636, -121.89004 37..."
2,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,Rapid 500,0,65850__60413,Santa Clara & 2nd__Santa Clara & 6th,"LINESTRING (-121.89017 37.33636, -121.89004 37..."
3,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,Rapid 500,0,65850__60413,Santa Clara & 2nd__Santa Clara & 6th,"LINESTRING (-121.88947 37.33669, -121.88844 37..."


### 5 variations

In [11]:
subset_df = dupes[dupes.n_geometry == 5].reset_index(drop=True)

for v in range(
    0, len(subset_df), int(len(subset_df)//4)
):
    plot_segment_variations(df, subset_df, v)

,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40132__40130,E Blithedale Ave & Roque Moraes Dr__E Blitheda...,"LINESTRING (-122.52774 37.90029, -122.52774 37..."
1,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40132__40130,E Blithedale Ave & Roque Moraes Dr__E Blitheda...,"LINESTRING (-122.52774 37.90029, -122.52774 37..."
2,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40132__40130,E Blithedale Ave & Roque Moraes Dr__E Blitheda...,"LINESTRING (-122.52774 37.90029, -122.52774 37..."
3,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40132__40130,E Blithedale Ave & Roque Moraes Dr__E Blitheda...,"LINESTRING (-122.52774 37.90029, -122.52774 37..."
4,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40132__40130,E Blithedale Ave & Roque Moraes Dr__E Blitheda...,"LINESTRING (-122.52774 37.90029, -122.52753 37..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,237,1,20048__10801,Zelzah / San Jose__Zelzah / Devonshire,"LINESTRING (-118.52340 34.26463, -118.52339 34..."
1,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,237,1,20048__10801,Zelzah / San Jose__Zelzah / Devonshire,"LINESTRING (-118.52336 34.26142, -118.52336 34..."
2,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,237,1,20048__10801,Zelzah / San Jose__Zelzah / Devonshire,"LINESTRING (-118.52340 34.26463, -118.52339 34..."
3,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,237,1,20048__10801,Zelzah / San Jose__Zelzah / Devonshire,"LINESTRING (-118.52336 34.26142, -118.52336 34..."
4,3f3f36b4c41cc6b5df3eb7f5d8ea6e3c,LA Metro Bus Schedule,all_day,237,1,20048__10801,Zelzah / San Jose__Zelzah / Devonshire,"LINESTRING (-118.52340 34.26463, -118.52339 34..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,7dbe3e19a4966e0c0531fa826e0446d8,San Joaquin Schedule,all_day,04,1,4205__3003,OAK ST & EL DORADO ST FS EB__FREMONT ST & CENT...,"LINESTRING (-121.29122 37.95780, -121.29124 37..."
1,7dbe3e19a4966e0c0531fa826e0446d8,San Joaquin Schedule,all_day,04,1,4205__3003,OAK ST & EL DORADO ST FS EB__FREMONT ST & CENT...,"LINESTRING (-121.29122 37.95780, -121.29124 37..."
2,7dbe3e19a4966e0c0531fa826e0446d8,San Joaquin Schedule,all_day,04,1,4205__3003,OAK ST & EL DORADO ST FS EB__FREMONT ST & CENT...,"LINESTRING (-121.29143 37.95856, -121.29145 37..."
3,7dbe3e19a4966e0c0531fa826e0446d8,San Joaquin Schedule,all_day,04,1,4205__3003,OAK ST & EL DORADO ST FS EB__FREMONT ST & CENT...,"LINESTRING (-121.29145 37.95868, -121.29133 37..."
4,7dbe3e19a4966e0c0531fa826e0446d8,San Joaquin Schedule,all_day,04,1,4205__3003,OAK ST & EL DORADO ST FS EB__FREMONT ST & CENT...,"LINESTRING (-121.29145 37.95868, -121.29133 37..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,19,1,679__680,Sunnymead FS Perris__Sunnymead at Bowling Alley,"LINESTRING (-117.22760 33.93923, -117.22765 33..."
1,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,19,1,679__680,Sunnymead FS Perris__Sunnymead at Bowling Alley,"LINESTRING (-117.22756 33.93917, -117.22765 33..."
2,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,19,1,679__680,Sunnymead FS Perris__Sunnymead at Bowling Alley,"LINESTRING (-117.22756 33.93917, -117.22765 33..."
3,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,19,1,679__680,Sunnymead FS Perris__Sunnymead at Bowling Alley,"LINESTRING (-117.22760 33.93923, -117.22765 33..."
4,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,19,1,679__680,Sunnymead FS Perris__Sunnymead at Bowling Alley,"LINESTRING (-117.22760 33.93923, -117.22781 33..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,85,0,64967__66045,Church & Pierce__Welburn & Church,"LINESTRING (-121.57944 37.02232, -121.57915 37..."
1,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,85,0,64967__66045,Church & Pierce__Welburn & Church,"LINESTRING (-121.57944 37.02232, -121.57915 37..."
2,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,85,0,64967__66045,Church & Pierce__Welburn & Church,"LINESTRING (-121.57944 37.02232, -121.57915 37..."
3,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,85,0,64967__66045,Church & Pierce__Welburn & Church,"LINESTRING (-121.57944 37.02232, -121.57915 37..."
4,fb467982dcc77a7f9199bebe709bb700,Bay Area 511 Santa Clara Transit Schedule,all_day,85,0,64967__66045,Church & Pierce__Welburn & Church,"LINESTRING (-121.57944 37.02232, -121.57915 37..."


### 10+ variations

In [12]:
subset_df = dupes[dupes.n_geometry >= 10].reset_index(drop=True)

for v in range(
    0, len(subset_df), int(len(subset_df)//4)
):
    plot_segment_variations(df, subset_df, v)

,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51391 37.92529, -122.51399 37..."
1,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51391 37.92529, -122.51399 37..."
2,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51386 37.92513, -122.51399 37..."
3,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51391 37.92529, -122.51399 37..."
4,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51386 37.92513, -122.51399 37..."
5,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51391 37.92529, -122.51399 37..."
6,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51391 37.92529, -122.51399 37..."
7,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51391 37.92529, -122.51399 37..."
8,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51391 37.92529, -122.51399 37..."
9,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,all_day,17,0,40296__40268,Hwy 101 @ Paradise Dr Bus Pad__Hwy 101 @ Lucky...,"LINESTRING (-122.51386 37.92513, -122.51399 37..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25874 34.04889, -118.25917 34..."
1,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25874 34.04889, -118.25920 34..."
2,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25857 34.04897, -118.25874 34..."
3,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25874 34.04889, -118.25982 34..."
4,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25874 34.04889, -118.25939 34..."
5,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25857 34.04897, -118.25874 34..."
6,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25874 34.04889, -118.25920 34..."
7,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25874 34.04889, -118.25939 34..."
8,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25874 34.04889, -118.25982 34..."
9,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,424005__485259,Flower St & 7th St__Flower St & 8th St,"LINESTRING (-118.25874 34.04889, -118.25917 34..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27018 34.03737, -118.27059 34..."
1,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27006 34.03756, -118.27018 34..."
2,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27015 34.03747, -118.27018 34..."
3,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27018 34.03737, -118.27108 34..."
4,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27018 34.03737, -118.27044 34..."
5,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27015 34.03747, -118.27018 34..."
6,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27006 34.03756, -118.27018 34..."
7,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27018 34.03737, -118.27044 34..."
8,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27018 34.03737, -118.27108 34..."
9,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,433918__433919,Figueroa St & Venice Blvd__Figueroa St & Washi...,"LINESTRING (-118.27018 34.03737, -118.27059 34..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25365 34.05421, -118.25375 34..."
1,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25347 34.05448, -118.25375 34..."
2,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25365 34.05419, -118.25375 34..."
3,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25358 34.05433, -118.25375 34..."
4,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25336 34.05457, -118.25375 34..."
5,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25365 34.05419, -118.25375 34..."
6,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25347 34.05448, -118.25375 34..."
7,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25336 34.05457, -118.25375 34..."
8,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25358 34.05433, -118.25375 34..."
9,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,4668076__4668077,Flower St & 3rd St (Southbound)__Flower St & 4...,"LINESTRING (-118.25365 34.05421, -118.25375 34..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26411 34..."
1,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26408 34..."
2,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26422 34..."
3,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26412 34..."
4,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26413 34..."
5,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26422 34..."
6,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26408 34..."
7,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26413 34..."
8,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26412 34..."
9,cc53a0dbf5df90e3009b9cb5d89d80ba,LA DOT Schedule,all_day,31,1,485260__433917,Flower St & 11th St__Figueroa St & Pico Blvd,"LINESTRING (-118.26403 34.04310, -118.26411 34..."


,schedule_gtfs_dataset_key,name,time_period,route_id,direction_id,stop_pair,stop_pair_name,geometry
0,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35773 33..."
1,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35778 33..."
2,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35778 33..."
3,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35773 33..."
4,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35773 33..."
5,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35778 33..."
6,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35778 33..."
7,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35774 33..."
8,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35789 33..."
9,d9d0325e50e50064e3cc8384b1751d67,Riverside Schedule,all_day,206,1,1041__1045,LE Outlet Center @ Park N Ride__Temescal @ Tom...,"LINESTRING (-117.35778 33.70471, -117.35774 33..."
